Notebook to create a sample of DP0.3 for different SSO populations

In [ ]:
from adler.objectdata.AdlerPlanetoid import AdlerPlanetoid
from adler.science.PhaseCurve import PhaseCurve
from adler.utilities.plotting_utilities import plot_errorbar
import adler.utilities.science_utilities as sci_utils

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import astropy.units as u
from lsst.rsp import get_tap_service
import sqlite3

In [ ]:
service = get_tap_service("ssotap")
assert service is not None

In [ ]:
# name of sample database
db_fname = "dp03_pop_sample.db"
cnx = sqlite3.connect(db_fname)

In [ ]:
n_samp = 100
pop_list = ["MB", "KBO", "NEA", "JT"]
# error_list = []

df_obj = pd.DataFrame()
for pop in pop_list:

    # load the sample objects (from ZTF)
    print(pop)
    fname_sample = "df_sample_{}_{}.csv".format(n_samp, pop)
    _df = pd.read_csv(fname_sample, index_col=0)

    # get the DP0.3 ssObjectIds for sample objects
    id_list = tuple(["2011 {}".format(x) for x in _df["sso_name"]])
    query = """
    SELECT
        mpc.fullDesignation, mpc.ssObjectId
    FROM
        dp03_catalogs_10yr.MPCORB as mpc
    WHERE
        mpc.fullDesignation in {}
    """.format(
        id_list
    )
    _df_obj = service.search(query).to_table().to_pandas()
    _df_obj["pop"] = pop
    print(len(_df_obj))

    df_obj = pd.concat([df_obj, _df_obj])

In [ ]:
df_obj

In [ ]:
df_obj.value_counts("pop")

In [ ]:
# query all tables for these objects and write them to a new database
ssoid_list = tuple(df_obj["ssObjectId"])
for tab in ["SSObject", "MPCORB", "DiaSource", "SSSource"]:
    print(tab)
    query = f"""
            SELECT
                *
            FROM
                dp03_catalogs_10yr.{tab}
            WHERE
                ssObjectId in {ssoid_list}
            """
    df_tab = service.search(query).to_table().to_pandas()
    df_tab.to_sql(tab, con=cnx, if_exists="replace", index=False)

In [ ]:
# test adler query
ssoid = ssoid_list[0]
planetoid1 = AdlerPlanetoid.construct_from_SQL(ssoid, sql_filename=db_fname)
fig = plot_errorbar(planetoid1, filt_list=planetoid1.filter_list, x_plot="phaseAngle", y_plot="reduced_mag")

In [ ]:
planetoid2 = AdlerPlanetoid.construct_from_RSP(ssoid)
fig = plot_errorbar(planetoid2, filt_list=planetoid2.filter_list, x_plot="phaseAngle", y_plot="reduced_mag")